In [1]:
from tensorflow import keras
import os, shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


이미지 파일 load

In [3]:
train_dir = '/content/drive/MyDrive/images_crawling/train'
test_dir='/content/drive/MyDrive/images_crawling/validation'

데이터 전처리

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255.)

Data Augmentation 정보 설정

In [6]:
train_datagen = ImageDataGenerator(
                            rescale=1./255.,
                            rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

test_datagen = ImageDataGenerator(
                            rescale=1./255.,
                          )
validation_datagen = ImageDataGenerator(
                            rescale=1./255.,
                          )

DirectoryIterator 객체 생성

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    shuffle = True,
    class_mode = 'categorical',
    subset = 'training'
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'categorical',
    subset = 'validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'categorical',
)

Found 300 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


CNN 모델 디자인 및 학습 정보 설정

In [8]:
from tensorflow.keras import models, layers, utils

In [9]:
model = models.Sequential([])

In [10]:

model.add(layers.Conv2D(filters=32, 
                        kernel_size=3, 
                        activation='relu',
                        input_shape=(150,150,3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=64, 
                        kernel_size=3, 
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=128, 
                        kernel_size=3, 
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=128, 
                        kernel_size=3, 
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))


model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))


In [11]:
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer='rmsprop',
              metrics=['accuracy'])

데이터 연결 및 학습

In [12]:
history = model.fit(train_generator,
          epochs=10,
          steps_per_epoch=len(train_generator),
          validation_data = validation_generator,
          validation_steps = len(validation_generator))

Epoch 1/10
15/15 [==============================] - 5s 178ms/step - loss: 1.3001 - accuracy: 0.4033
Epoch 2/10
15/15 [==============================] - 3s 186ms/step - loss: 0.7684 - accuracy: 0.5800
Epoch 3/10
15/15 [==============================] - 3s 184ms/step - loss: 0.7565 - accuracy: 0.6400
Epoch 4/10
15/15 [==============================] - 3s 188ms/step - loss: 0.6436 - accuracy: 0.6833
Epoch 5/10
15/15 [==============================] - 3s 186ms/step - loss: 0.6050 - accuracy: 0.7100
Epoch 6/10
15/15 [==============================] - 3s 187ms/step - loss: 0.5803 - accuracy: 0.7233
Epoch 7/10
15/15 [==============================] - 3s 186ms/step - loss: 0.5335 - accuracy: 0.7700
Epoch 8/10
15/15 [==============================] - 3s 186ms/step - loss: 0.5596 - accuracy: 0.6933
Epoch 9/10
15/15 [==============================] - 3s 187ms/step - loss: 0.6233 - accuracy: 0.6967
Epoch 10/10
15/15 [==============================] - 3s 185ms/step - loss: 0.4740 - accuracy: 0.7733

In [13]:
!mkdir -p saved_model
model.save('saved_model/image_sentimental_analysis')

INFO:tensorflow:Assets written to: saved_model/image_sentimental_analysis/assets


test 데이터셋으로 모델 성능 평가

In [ ]:
loss, accuracy = model.evaluate(test_generator)